# PHYS 2600 - Final Project 2: `gravity`

## Motion of Small Objects in the Solar System

This is one possible _final project_; __you must choose one (and only one) project to complete and hand in.__  

The deadline for this project is __12:00 midnight (Boulder time) on Monday, December 17.__  Since final grades must be submitted to the university shortly after the end of the exam period, to allow for timely grading _no late submissions will be accepted._

You are permitted to discuss the final projects with other students in the class,  including both the physics and computing aspects, but __your submitted project must be entirely your own__; no direct copying of Python code is permitted, and you must write up the presentation in your own words.

When you submit your project, you must provide __two (2) files__:

* `gravity.py`: a Python module ([see lecture 20](https://physicscourses.colorado.edu/phys2600/phys2600_fa18/lecture/lec20-modules-files/)) which implements the functions listed in the Application Progamming Interface (API) below.
* `gravity_presentation.ipynb`: a Jupyter notebook which uses the code from your Python module to answer the physics questions.  Use of Markdown text, MathJax equations, and plots are all encouraged to make your presentation clear and compelling!


The rubric for grading will be split 50/50 between the code (in your `.py` module and Jupyter notebook) and the response to the physics questions (provided in `gravity_presentation.ipynb`):
* Basic functionality of code (passes provided tests, follows API specification): __30%__
* __Six (6)__ additional API tests _you write_ (tests are provided and are correct and useful): __10%__
* Documentation of code (docstrings and comments make the code clear): __10%__
* Correct and detailed answers to physics questions: __40%__
* Quality of presentation (clear and readable, appropriate use of Markdown, equations, and plots): __10%__

A _bonus of up to 10%_ is available for the extra-credit "challenge" physics question at the end.  (These challenge questions are meant to be really hard!  But partial credit will be awarded if you make some progress on them.)

# Overview

The motion of large objects in the solar system is very well understood: Kepler's laws of planetary motion are one of the first applications of the law of gravitation that we all learn in intro physics.  However, we know that gravity becomes complicated quickly: even though the motion of _two_ objects can be solved on pen and paper, the "three-body problem" (motion of three massive objects under gravity) already has no analytic solutions.  So tracking a single object (or even several) as it moves through the gravitational field of the solar system is not doable with pen and paper - making it a great candidate for numerical study.

For this project, you'll begin by modeling the Sun and the planets of the solar system in their known orbits.  Then, you will write a code which can solve for the motion of a small object through the complex gravity field of the Sun and planets.  The main physics goals will involve looking at incoming asteroids and comets to see if they pose a threat of a collision with the Earth, and studying the motion of 'Oumuamua - the first interstellar object recorded passing through our solar system.


# Physics background and important equations

Everything starts with the law of gravitation: the force on a small mass $m$ due to large mass $M$ at the origin is

$$
\mathbf{F}_g = -\frac{GmM}{r^3} \mathbf{r}
$$

where $\mathbf{r} = (x,y,z)$ is the position vector of $m$ with respect to $M$, and the minus sign appears since gravity is attractive (so the force is always towards the origin.)  

For this project, we'll have multiple sources of gravity to worry about, so this is better written in the more general form (cancelling the mass $m$ out)

$$
\ddot{\mathbf{r}} = -\frac{GM}{|\mathbf{r} - \mathbf{R}|^3} (\mathbf{r} - \mathbf{R})
$$

where $\mathbf{r}$ is the position vector of the test mass $m$, and $\mathbf{R}$ is the position vector of the gravity source $M$.  The double dots denote two time derivatives, i.e. $\ddot{\mathbf{r}} = d^2 \mathbf{r} / dt^2$.

### Model of the solar system

Before we can study the motion of small objects, we need a model for the solar system itself.  Assuming all the planets are much lighter than the Sun, the planets follow elliptical orbits around the Sun of the form

$$
r(\phi) = \frac{a (1 - \epsilon^2)}{1+\epsilon \cos (\phi - \omega)}
$$
where $a$ is the semimajor axis of the orbit, $\epsilon$ is the eccentricity (how un-circular the orbit is), and $\phi$ is the orbital phase.  The angle $\omega$ determines the location of __perihelion__ (closest approach to the Sun), and is different for every planet.  

To determine $\phi$ as a function of time, we just use Kepler's second law, which can be written as

$$
\frac{d\phi}{dt} = \frac{L_z}{\mu r^2} \approx \frac{\sqrt{GM_{\odot} a (1 - \epsilon^2)}}{r^2}
$$

where $M_{\odot}$ is the mass of the Sun,

$$
M_{\odot} = 1.9885 \times 10^{30}\ {\rm kg}.
$$

For non-circular orbits the equation for $d\phi / dt$ leads to tricky integrals that can't be done in closed form, but we don't care since we're doing numerics: we'll just use this equation directly to update $\phi$ at each timestep, given the current value of $r$.  Assuming all eight planets are in the same plane (which is a good approximation: Mercury is 7 degrees out of the ecliptic plane, but it's also a small planet!), their coordinates as a function of time are then just

$$
\mathbf{R} = (r \cos \phi, r \sin \phi, 0)
$$


| planet | mass ($\times 10^{24}$ kg) |  a (AU) | $\epsilon$ | $\omega$ (${}^{\circ}$) | $\phi_0$ (${}^{\circ}$) [11/22/18] |
|--------|------|-----|------------|-------------------|-----|
| Mercury | 0.330104 | 0.38709843 | 0.20563661 | 77.45771895 | 19.08833658 |
| Venus | 4.86732 | 0.72332102 | 0.00676399 | 131.76755713 | 286.68776598 |
| Earth | 5.97219 | 1.00000018 | 0.01673163 | 102.93005885 | 331.16416467 |
| Mars | 0.641693 | 1.52371243 | 0.09336511 | -23.91744784 | 326.20022408 |
| Jupiter | 1898.13 | 5.20248019 | 0.04853590 | 14.27495244 | 242.48991503 |
| Saturn | 568.319 | 9.54149883 | 0.05550825 | 92.86136063 | 281.55831164 |
| Uranus | 86.8103 | 19.18797948 | 0.04685740 | 172.43404441 | 29.32733579 |
| Neptune | 102.41 | 30.06952752 | 0.00895439 | 46.68158724 | 351.14513741 |

Orbital parameters are [taken from NASA here](https://ssd.jpl.nasa.gov/txt/aprx_pos_planets.pdf) - see also the machine-readable data table [here](https://ssd.jpl.nasa.gov/txt/p_elem_t2.txt).  Masses are from NASA's [Solar System Exploration site](https://solarsystem.nasa.gov/planet-compare/).  _(Note: I did some coordinate conversions myself to put this into our coordinate system.  No warranty is given that these coordinates will perfectly match other sources you may find.)_

We can also compute the orbital parameters for Pluto, although you __should not include Pluto in your model of the solar system__ when answering the questions below; one of the questions deals with Pluto itself, so knowing its orbit may be useful.  I've also given you the orbits of two other objects of interest, the famous Halley's Comet and the interstellar visitor 'Oumuamua.

| object | a (AU) | $\epsilon$ | $\omega$ (${}^{\circ}$) | $\phi_0$ (${}^{\circ}$) |
|--------|--------|------------|-------------------------|-------------------------|
| Pluto  | 39.48211675 | 0.24882730 | 224.06891629 | 265.9093415 |
| Halley's Comet | 17.834 | 0.96714 | 111.33 | --- |
| 'Oumuamua | 0.25383 | 1.1956 | 241.43 | --- |

# Computational Strategy and Algorithms

We have two separate models to keep track of.  For the model of the solar system, we calculate a numerical value for $d\phi / dt$ for each planet.  Given a timestep $dt$, we can use the technique of [Lecture 18](https://physicscourses.colorado.edu/phys2600/phys2600_fa18/lecture/lec18-discrete-derivs/) to update the angular positions $\phi$, and then compute $r(\phi)$ from above.

For the motion of the small mass itself, since we have a single second-order equation, we use the familiar trick of splitting it into two equations:

$$
\mathbf{v} = \frac{d\mathbf{r}}{dt} \\
\frac{d\mathbf{v}}{dt} = \ddot{\mathbf{r}} = (...)
$$

(Actually, it's six equations since $\mathbf{r}$ and $\mathbf{v}$ are both three-vectors, but it's the same trick.)  Then we use the methods of Lecture 18 to update both position and velocity for every time step $dt$.

That's all we need for the basic functionality of this project!  The computational setup here is relatively simple for a single object - keeping track of many objects at once and doing data analysis on the outputs are where the difficulty lies...

# Application Programming Interface (API) specification

__You must implement all of the below functions, according to the specifications given.__  I actually recommend _not_ using Unyt for the computationally intensive parts of this program: it will slow down the code significantly!  Instead, you should use Unyt to calculate the numerical values you need in a certain unit system, and then work with pure NumPy arrays from there.  I used "kg-AU-s" units, but you can choose what you like.

A commonly-used tuple structure appearing in the API is the __planet__:

```
my_planet = (M, a, eps, phi, omega)
```

Here `M` is the mass of the planet, `phi` is the current angular position of the planet in our coordinates, and `a`, `eps` and `omega` are orbital parameters of the planet as described above.


### Planets:

* `get_planet_r(planet)`: Given a planet tuple, returns its current distance from the Sun $r(\phi)$.
* `get_planet_coords(planet)`: Given a planet tuple, returns its current three-dimensional Cartesian coordinates $(x,y,z)$.
* `get_planet_orbit(planet, phi_linspace)`: Given a planet tuple and a NumPy array of phi values, returns a tuple of two arrays `(x_array, y_array)` containing the $x$ and $y$ coordinates of the planet at each phi value.  (This is useful for plotting orbits.)
* `update_planet_position(planet, dt)`: Given a planet tuple and a timestep `dt`, returns a new planet tuple with the planet's position `phi` updated.
* `update_all_planets(planets, dt)`: Given a list of planet tuples and a timestep `dt`, updates the positions of _all_ the planets, and returns them as a list.

### Small mass:

* `accel_g_sun(vec_r)`: Given a position three-vector `vec_r` = $(x,y,z)$, computes and returns the gravitational acceleration due to the Sun as a (NumPy) three-vector.
* `accel_g_planet(vec_r, planet)`: Given a position three-vector `vec_r` and a planet tuple, computes and returns the gravitational acceleration due to that planet as a (NumPy) three-vector.
* `update_position(vec_r, vec_v, planets, dt)`: Given the current position vector `vec_r` and velocity vector `vec_v`, a list of all planets, and a timestep `dt`, computes and returns updated position and velocity vectors `(vec_r_new, vec_v_new)` due to gravity from the planets and the Sun.

### Other functions:

* `get_planet_distances(vec_r, planets)`: Given a position vector `vec_r` and a list of planets, returns a NumPy array containing the distances from `vec_r` to each planet (in the same order as in `planets`).
* `run_API_tests()`: A custom function that should use assertions to test the other functions implemented in the API.  If all tests are passed, the function should simply return the `None` object.  You should implement __at least six (6) tests__ inside this function on your own; additional tests will be provided after the checkpoint is due on Friday 12/7.

## Main loop

The below code implements the "main loop", finding the trajectory of a small object given its initial position `vec_r0`, initial velocity `vec_v0`, a list of all planet tuples to use (at their initial positions), and a NumPy linspace `t_steps` containing the full time range to simulate over, separated by the desired timestep `dt`.

Once you have implemented the API above, add this code to your `gravity.py` module, then call it to run the simulation in your notebook!

In [ ]:
def find_trajectory(vec_r0, vec_v0, planets, t_steps):
    """
    Main loop for solar system gravitation project.
    
    Arguments:
    =====
    * vec_r0: Initial 3-vector position of the small mass (Cartesian coordinates.)  
    * vec_v0: Initial 3-vector velocity of the small mass (Cartesian coordinates.)
    * planets: a list of planet tuples, at their initial positions.
        A planet tuple has the form:
            (M, a, eps, phi, omega)
        where M is the planet's mass, phi is the planet's angular position, 
        and a, eps, omega are orbital parameters.
    * t_steps: NumPy array (linspace or arange) specifying the range of times to simulate
        the trajectory over, regularly spaced by timestep dt.
        
    Returns:
    =====
    A tuple of the form (traj, planet_distance).
    
    "traj" contains the coordinates (x,y,z) of the test mass at each 
    corresponding time in t_steps, as a (3) x (Nt) array.
    "planet_distance" contains the distances from the small mass
    to each planet in planets, in order, as a function of time - this is a
    (len(planets)) x (Nt) array.
    
    Example usage:  (using kg-AU-s units)
    =====
    >> import unyt as u
    >> earth = (5.97219e24, 1.0, 0.01673163, 5.779905, 1.88570)
    >> r0 = np.array([-0.224, 0.98, 0.0])  # AU
    >> v0 = np.array([2e-9, 0.0, 0.0]) # AU/s 
    >> t = (np.arange(0, 4*365) * u.day).to_value('s')  # evolve for 4 years
    >> traj, pd = find_trajectory(r0, v0, [earth], t)
    
    """
    
    dt = t_steps[1] - t_steps[0]
    Nt = len(t_steps)
    
    traj = np.zeros((3, Nt))
    traj[:,0] = vec_r0
    
    planet_distance = np.zeros((len(planets), Nt))
    planet_distance[:,0] = get_planet_distances(vec_r0, planets)
    
    vec_v = vec_v0
    
    for i in range(Nt-1):
        (traj[:,i+1], vec_v) = update_position(traj[:,i], vec_v, planets, dt)        
        planets = update_all_planets(planets, dt)
        planet_distance[:,i+1] = get_planet_distances(traj[:,i+1], planets)
        
    return (traj, planet_distance)